In [1]:
using EDM4hep
using EDM4hep.RootIO
# using JetReconstruction
using StaticArrays
using LorentzVectorHEP
using JSON
using ONNXRunTime
# using LinearAlgebra
using StructArrays
using UnROOT
# using Plots

In [2]:
using Pkg
Pkg.resolve()
Pkg.develop(Pkg.PackageSpec(path = "/Users/harrywanghc/Developer/2025/HarryFork/JetReconstruction.jl"))
using JetReconstruction
Pkg.status()

  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Warning: Replacing docs for `JetReconstruction.build_constituents :: Tuple{StructVector{ReconstructedParticle}, Vector{ReconstructedParticle}}` in module `JetReconstruction`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `JetReconstruction.build_constituents_cluster :: Tuple{StructVector{ReconstructedParticle}, Vector{Vector{Int64}}}` in module `JetReconstruction`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `JetReconstruction.get_jet_constituents :: Tuple{Vector{StructVector{ReconstructedParticle}}, Int64}` in module `JetReconstruction`
└ @ Base.Docs docs/Doc

Status `~/.julia/environments/v1.11/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [eb32b910] EDM4hep v0.4.3
  [5c1252a2] GeometryBasics v0.5.7
  [7073ff75] IJulia v1.27.0
  [682c06a0] JSON v0.21.4
  [44e8cb2c] JetReconstruction v0.4.9 `~/Developer/2025/HarryFork/JetReconstruction.jl`
  [98e50ef6] JuliaFormatter v2.1.2
  [f612022c] LorentzVectorHEP v0.1.6
  [e034b28e] ONNXRunTime v1.3.1
  [91a5bcdd] Plots v1.40.13
  [90137ffa] StaticArrays v1.9.13
  [10745b16] Statistics v1.11.1
⌅ [09ab397b] StructArrays v0.6.21
  [1e6cf692] TestEnv v1.102.0
  [3cd96dde] UnROOT v0.10.36
  [37e2e46d] LinearAlgebra v1.11.0
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [3]:
# Paths to model files
model_dir = "data/wc_pt_7classes_12_04_2023"
onnx_path = joinpath(model_dir, "fccee_flavtagging_edm4hep_wc_v1.onnx")
json_path = joinpath(model_dir, "fccee_flavtagging_edm4hep_wc_v1.json")

# Load the configuration and model
config = JSON.parsefile(json_path)
model = ONNXRunTime.load_inference(onnx_path)

# Display the output classes we'll predict
println("The model predicts these flavor classes:")
for class_name in config["output_names"]
    println(" - ", class_name)
end

The model predicts these flavor classes:
 - recojet_isG
 - recojet_isQ
 - recojet_isS
 - recojet_isC
 - recojet_isB


In [4]:
# Path to ROOT file with EDM4hep data
edm4hep_path = "data/events_080263084.root"
reader = RootIO.Reader(edm4hep_path)

# Get event information
events = RootIO.get(reader, "events")
println("Loaded $(length(events)) events")

# Choose a specific event to analyze (event #13)
event_id = 13
evt = events[event_id]
println("Processing event #$event_id")

# Get reconstructed particles and tracks
recps = RootIO.get(reader, evt, "ReconstructedParticles")
tracks = RootIO.get(reader, evt, "EFlowTrack_1")

# Set the magnetic field strength (in Tesla)
bz = 2.0  # This should be obtained from your data if possible

# Get other needed collections for feature extraction
trackdata = RootIO.get(reader, evt, "EFlowTrack")
trackerhits = RootIO.get(reader, evt, "TrackerHits")
gammadata = RootIO.get(reader, evt, "EFlowPhoton")
nhdata = RootIO.get(reader, evt, "EFlowNeutralHadron")
calohits = RootIO.get(reader, evt, "CalorimeterHits")
dNdx = RootIO.get(reader, evt, "EFlowTrack_2")

# Load track length information from the ROOT file
# Note: This is a placeholder. You need to replace this with the loading from EDM4hep.jl once I fix the bug in EDM4hep.
f = ROOTFile(edm4hep_path)
mytree = LazyTree(f, "events", ["EFlowTrack_L"])
track_L = collect(mytree[event_id].EFlowTrack_L)

println("Loaded $(length(recps)) reconstructed particles")
println("Loaded $(length(tracks)) tracks")

Loaded 100000 events
Processing event #13
Loaded 107 reconstructed particles
Loaded 44 tracks


In [ ]:
# include("/Users/harrywanghc/Developer/2025/HarryFork/JetReconstruction.jl/src/ReconstructedParticle.jl")
# # Find muons in the event
# muons = RootIO.get(reader, evt, "Muon#0")
# muons_all = ReconstructedParticle.get(muons, recps)
# muons_selected = ReconstructedParticle.sel_p(20.0)(muons_all)
# # TODO: There is a bug in the sel_p function. 
# # When there's only one muon, it should returns a single value instead of a collection but it's buggy.

# # Remove high-momentum muons from the input particles
# recps_no_muons = ReconstructedParticle.remove(recps, muons_selected)
# vrecps = collect(recps_no_muons)

# # Cluster jets using the EEkt algorithm with R=2.0 and p=1.0
# cs = jet_reconstruct(vrecps; p = 1.0, R = 2.0, algorithm = JetAlgorithm.EEKt)

# # Get 2 exclusive jets
# jets = exclusive_jets(cs; njets=2, T=EEJet)

# # For each jet, get its constituent particles
# constituent_indices = [constituent_indexes(jet, cs) for jet in jets]
# jet_constituents = build_constituents_cluster(recps_no_muons, constituent_indices)


In [5]:
# Cluster jets using the EEkt algorithm with R=2.0 and p=1.0
cs = jet_reconstruct(recps; p = 1.0, R = 2.0, algorithm = JetAlgorithm.EEKt)

# Get 2 exclusive jets
jets = exclusive_jets(cs; njets=2, T=EEJet)

# For each jet, get its constituent particles
constituent_indices = [constituent_indexes(jet, cs) for jet in jets]
jet_constituents = build_constituents_cluster(recps, constituent_indices)


2-element Vector{StructVector{ReconstructedParticle}}:
 [ReconstructedParticle(#76, 22, 0.1520507f0, (-0.089623965, -0.118329994, 0.032938913), (0.0, 0.0, 0.0), 0.0f0, 3.1475472f-5, 0.0f0, Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Cluster#[32], Track#[], ReconstructedParticle#[], ParticleID#[], #0, #0), ReconstructedParticle(#75, 22, 0.38633013f0, (-0.3512648, 0.14457734, 0.07043707), (0.0, 0.0, 0.0), 0.0f0, 9.40061f-5, 0.0f0, Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Cluster#[31], Track#[], ReconstructedParticle#[], ParticleID#[], #0, #0), ReconstructedParticle(#78, 22, 0.049976002f0, (-0.041344654, -0.018857136, 0.020799734), (0.0, 0.0, 0.0), 0.0f0, -9.3014905f-6, 0.0f0, Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Cluster#[34], Track#[], ReconstructedParticle#[], ParticleID#[], #0, #0), ReconstructedParticle(#45, 22, 0.13387372f0, (-0.039546303, 0.03148594, -0.1239633), (0.0, 0.0, 0.0), 0.0f0, -4.3123426f-5, 0.0f0, Float32[0.0, 0.0, 0.

In [6]:
println("Extracting features for flavor tagging...")
feature_data = extract_features(
    jets, 
    jet_constituents, 
    tracks, 
    bz, 
    track_L, 
    trackdata, 
    trackerhits, 
    gammadata, 
    nhdata, 
    calohits, 
    dNdx
)
println("Step 1: Feature extraction completed.")

model, config = setup_weaver(
    onnx_path,
    json_path
)

println("Step 2: Weaver setup completed.")

input_tensors = prepare_input_tensor(
    jet_constituents,
    jets,
    config,
    feature_data
)

println("Step 3: Input tensor preparation completed.")

println("Running flavor tagging inference...")
weights = get_weights(
    0,  # Thread slot
    feature_data,
    jets,
    jet_constituents,
    config,
    model
)
println("Step 4: Weights retrieval completed.")

jet_scores = Dict{String, Vector{Float32}}()
for (i, score_name) in enumerate(config["output_names"])
    jet_scores[score_name] = get_weight(weights, i-1)
end

println("Jet scores:")
for (name, scores) in jet_scores
    println(" - $name: $(scores[1])")
end

Extracting features for flavor tagging...
Step 1: Feature extraction completed.
Step 2: Weaver setup completed.
Step 3: Input tensor preparation completed.
Running flavor tagging inference...
Step 4: Weights retrieval completed.
Jet scores:
 - recojet_isG: 0.677954
 - recojet_isB: 0.017783705
 - recojet_isQ: 0.06406556
 - recojet_isC: 0.011805316
 - recojet_isS: 0.2283914
